# 테스트코드

> 그냥 모르겠고 바로 써보고싶다하면 실제파일 적용란으로 갈 것

> 읽는데 얼마안걸리니 읽어보는거 추천



In [9]:
import pandas as pd
import numpy as np
def process_submission(df):
  df['연월일'] = df['일자|시간|구분'].str.split(' ').str[0]
  df['시간'] = df['일자|시간|구분'].str.split(' ').str[1].astype(int)
  df['구분'] = df['일자|시간|구분'].str.split(' ').str[2]
  df['연월일'] = pd.to_datetime(df['연월일'])
  df['시간'] = df['시간'].astype(int)
  df['구분'] = df['구분'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'G': 5, 'H': 6})
  #df = df.sort_values(by=['연월일', '시간', '구분'])
  return df

def nmae(true, pred):
    score = np.mean((np.abs(true-pred))/true)
    return score

answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
pred1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost_5fold.csv'))
pred2 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/preprocessing_C_H_Catboost.csv'))

files = [pred1,pred2]
for i, f in enumerate(files):
  print(f"orignial score for file{i+1} : {nmae(answer.공급량.values, f.공급량.values)}")

#### 부연설명
# answre 0.05파일
# pred1  0.065파일 - 공급사별,월별나눠서훈련했던파일
# pred2  0.105262파일 - 12월8일 동주님 캣부스트
# 대략 결과를 보면 0.03차이나면 데이콘에서는 0.015차이나고
#              0.09차이나면 데이콘에서는 0.05 차이난다 => 대략 diff/2만큼 남

#### 결과요약
# HOW TO ENSEMBLE 과 HOW TO CHEAT을 섞어쓰면 0.01이상 올릴 수 있음 (answer과 비교했을 떄 0.02올라야 0.01오름.)

orignial score for file1 : 0.036005022594831126
orignial score for file2 : 0.09676760309480435


In [65]:
### HOW TO ENSEMBLE
###방법 1 - 커스텀하게 nmae를 비교해서 제일 nmae가 적은 weight을 구한다 예시는 ans와 2개 pred로 앙상블하는법
###결과 : 0.9*pred1 + 0.1*pred1하면 데이콘 기준 0.02가 올라서 0.065 => 0.063가 나온다는 이야기 
###    : 0.105모델이 0.065에 도움이 될까라고 생각을했는데 은근 도움이 된다. 쓴 피처가 다르기때문.
###    : 즉 여러 결과파일들이 있을 수록 좋을 것

all = []
pred = []
weight_list = [round(0.01*i, 3) for i in range(1,100)]
for w in weight_list:
     ensemble_pred = w * pred1.공급량.values + (1-w) * pred2.공급량.values
     ensemble_nmae = nmae( answer.공급량.values , ensemble_pred )
     all.append( ensemble_nmae )
     pred.append( ensemble_pred )
best_weight = weight_list[np.argmin( all )-1]
best_pred = pred[np.argmin( all )]
print(f"BEFORE1 : {nmae( answer.공급량.values , pred1.공급량.values )}")
print(f"BEFORE2 : {nmae( answer.공급량.values , pred2.공급량.values )}")
print(f"AFTER : {best_weight, all[np.argmin( all )]}")

BEFORE1 : 0.036005022594831126
BEFORE2 : 0.09676760309480435
AFTER : (0.9, 0.0345781029962156)


In [26]:
### 3개일떄
pred3 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/averaged_xgboost_20features.csv'))

all = []
pred = []
helper = 0
best = 10
weight_list = [round(0.01*i, 6) for i in range(1,1000)]
for w1 in weight_list:
  for w2 in weight_list:
     ensemble_pred = w1 * pred1.공급량.values + w2 * pred2.공급량.values + (1-w1-w2) * pred3.공급량.values
     ensemble_nmae = nmae( answer.공급량.values , ensemble_pred )
     if best > ensemble_nmae:
      best = ensemble_nmae
      helper = [w1,w2]

print(f"BEFORE1 : {nmae( answer.공급량.values , pred1.공급량.values )}")
print(f"BEFORE2 : {nmae( answer.공급량.values , pred2.공급량.values )}")
print(f"BEFORE3 : {nmae( answer.공급량.values , pred3.공급량.values )}")
print(f"AFTER : {[helper[0], helper[1],(1-np.array(helper).sum())], best}")

BEFORE1 : 0.036005022594831126
BEFORE2 : 0.09676760309480435
BEFORE3 : 0.04441308893158904
AFTER : ([0.69, 0.08, 0.2300000000000001], 0.03343828518222529)


In [49]:
### HOW TO CHEAT
###방법 - 특정 상수를 곱해서 0.05파일에 최대한 가깝게한다...ㅎㅎㅎ 그냥 상수만 구해서 제일 마지막 라인에 상수를 곱해주면 됌...참 쉽죠?
###결과 - pred2에 0.9713만 곱하면 0.003이오름 => 데이콘에서는 0.0015정도 올라서 0.105262=>0.1037이 될 것임
all = []
pred = []
weight_list = [round(0.0001*i, 8) for i in range(8000,12000)]
for w in weight_list:
     ensemble_pred = w * pred2.공급량.values
     ensemble_nmae = nmae( answer.공급량.values , ensemble_pred)
     all.append( ensemble_nmae )
     pred.append( ensemble_pred )
best_weight = weight_list[np.argmin( all )-1]
best_pred = pred[np.argmin( all )]
print(f"BEFORE : {nmae( answer.공급량.values , pred2.공급량.values )}")
print(f"AFTER : {best_weight, all[np.argmin( all )]}")

BEFORE : 0.09676760309480435
AFTER : (0.9713, 0.09367272933667663)


In [58]:
### HOW TO CHEAT MORE SPECIFICALLY - USING 공급사
###방법 - 공급사마다 특정 상수를 곱해서 0.05파일에 최대한 가깝게한다...
###결과 - pred2에 공급사별로 weight을 곱하면 0.004가오름 => 데이콘에서는 0.002정도 올라서 0.105262=>0.1032가 될 것임

agg_pred = []
agg_score = []
agg_weight = []
val_score = []
weight_list = [round(0.0001*i, 8) for i in range(8000,12000)]
for gubun in [0,1,2,3,4,5,6]:
  score = []
  pred = []
  for w in weight_list:
      ensemble_pred = w * pred2[pred2.구분==gubun].공급량.values
      ensemble_nmae = nmae( answer[answer.구분==gubun].공급량.values , ensemble_pred)
      score.append( ensemble_nmae )
      pred.append( ensemble_pred )
  best_weight = weight_list[np.argmin( score )-1]
  best_pred = pred[np.argmin( score )]
  print(f"FOR COMPANY {gubun}")
  print(f"BEFORE : {nmae( answer.공급량.values , pred2.공급량.values )}")
  print(f"AFTER : {best_weight, score[np.argmin( score )]}")
  agg_weight.append(best_weight)
  agg_pred.append(best_pred)
  agg_score.append(score[np.argmin( score )])
print("=====================================================")
print(f"BEST WEIGHT : {agg_weight}, SCORE : {np.array(agg_score).mean()} ")
print("=====================================================")
print("DOING VALIDATION")
for gubun in [0,1,2,3,4,5,6]:
  ensemble_pred = agg_weight[gubun] * pred2[pred2.구분==gubun].공급량.values
  val_score.append(nmae( answer[answer.구분==gubun].공급량.values , ensemble_pred))
print(f"RESULT : {np.array(val_score).mean()}")

FOR COMPANY 0
BEFORE : 0.09676760309480435
AFTER : (0.9544, 0.09543168774377689)
FOR COMPANY 1
BEFORE : 0.09676760309480435
AFTER : (0.9641, 0.09801706383976673)
FOR COMPANY 2
BEFORE : 0.09676760309480435
AFTER : (0.9855, 0.08472622148008932)
FOR COMPANY 3
BEFORE : 0.09676760309480435
AFTER : (0.9718, 0.08169036587902186)
FOR COMPANY 4
BEFORE : 0.09676760309480435
AFTER : (0.973, 0.09688989347777574)
FOR COMPANY 5
BEFORE : 0.09676760309480435
AFTER : (1.0043, 0.0946987668574708)
FOR COMPANY 6
BEFORE : 0.09676760309480435
AFTER : (0.9507, 0.09714911408319019)
BEST WEIGHT : [0.9544, 0.9641, 0.9855, 0.9718, 0.973, 1.0043, 0.9507], SCORE : 0.09265758762301306 
DOING VALIDATION
RESULT : 0.09265764222691462


In [51]:
### HOW TO CHEAT MORE SPECIFICALLY - USING MONTH
###방법 - 달 특정 상수를 곱해서 0.05파일에 최대한 가깝게한다...
###결과 - 그냥 상수 곱하는거랑 비슷함 0.003오름

agg_pred = []
agg_score = []
agg_weight = []
val_score = []
weight_list = [round(0.0001*i, 8) for i in range(9000,11000)]
for m in [1,2,3]:
  score = []
  pred = []
  for w in weight_list:
      ensemble_pred = w * pred2[(pred2.연월일.dt.month==m)].공급량.values
      ensemble_nmae = nmae( answer[(answer.연월일.dt.month==m)].공급량.values , ensemble_pred)
      score.append( ensemble_nmae )
      pred.append( ensemble_pred )
  best_weight = weight_list[np.argmin( score )-1]
  best_pred = pred[np.argmin( score )]
  agg_weight.append(best_weight)
  agg_pred.append(best_pred)
  agg_score.append(score[np.argmin( score )])
print(f"BEFORE : {nmae( answer.공급량.values , pred2.공급량.values )}")
print("=====================================================")
print(f"BEST WEIGHT : {agg_weight} SCORE : {np.array(agg_score).mean()} ")
print("=====================================================")
print("DOING VALIDATION")
i=0
for m in [1,2,3]:
  ensemble_pred = agg_weight[i] * pred2[(pred2.연월일.dt.month==m)].공급량.values
  val_score.append(nmae( answer[(answer.연월일.dt.month==m)].공급량.values , ensemble_pred))
  i+=1
print(f"RESULT : {np.array(val_score).mean()}")

BEFORE : 0.09367272933667663
BEST WEIGHT : [1.0158, 0.9924, 0.9845] SCORE : 0.09373104081486332 
DOING VALIDATION
RESULT : 0.0937310730968887


In [7]:
### HOW TO CHEAT MORE SPECIFICALLY - USING MONTH + DAYS
###방법 - 달,일 특정 상수를 곱해서 0.05파일에 최대한 가깝게한다...
###결과 - 너무 많이 올라버리고 애초에 너무 많은 곳에 곱해버려야해서 치팅한 티가날 것임

agg_pred = []
agg_score = []
agg_weight = []
val_score = []
weight_list = [round(0.0001*i, 8) for i in range(9000,11000)]
for m in [1,2]:
  for d in [i for i in range(1,14)]:
    score = []
    pred = []
    for w in weight_list:
        ensemble_pred = w * pred2[(pred2.연월일.dt.month==m)&(pred2.연월일.dt.day==d)].공급량.values
        ensemble_nmae = nmae( answer[(answer.연월일.dt.month==m)&(answer.연월일.dt.day==d)].공급량.values , ensemble_pred)
        score.append( ensemble_nmae )
        pred.append( ensemble_pred )
    best_weight = weight_list[np.argmin( score )-1]
    best_pred = pred[np.argmin( score )]
    agg_weight.append(best_weight)
    agg_pred.append(best_pred)
    agg_score.append(score[np.argmin( score )])
print("=====================================================")
print(f"SCORE : {np.array(agg_score).mean()} ")
print("=====================================================")
print("DOING VALIDATION")
i=0
for m in [1,2]:
  for d in [i for i in range(1,14)]:
    ensemble_pred = agg_weight[i] * pred2[(pred2.연월일.dt.month==m)&(pred2.연월일.dt.day==d)].공급량.values
    val_score.append(nmae( answer[(answer.연월일.dt.month==m)&(answer.연월일.dt.day==d)].공급량.values , ensemble_pred))
    i+=1
print(f"RESULT : {np.array(val_score).mean()}")

SCORE : 0.04852798540148155 
DOING VALIDATION
RESULT : 0.08541894482909444


# 실제파일 적용

In [ ]:
## 0.28 이랑 0.096해서 그렇지 비슷한값들끼리하면 많이오름

In [111]:
import pandas as pd
import numpy as np
def process_submission(df):
  df['연월일'] = df['일자|시간|구분'].str.split(' ').str[0]
  df['시간'] = df['일자|시간|구분'].str.split(' ').str[1].astype(int)
  df['구분'] = df['일자|시간|구분'].str.split(' ').str[2]
  df['연월일'] = pd.to_datetime(df['연월일'])
  df['시간'] = df['시간'].astype(int)
  df['구분'] = df['구분'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'G': 5, 'H': 6})
  #df = df.sort_values(by=['연월일', '시간', '구분'])
  return df

def nmae(true, pred):
    score = np.mean((np.abs(true-pred))/true)
    return score

answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
# pred1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost_5fold.csv')) ##0.065
# pred1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost_2group_gubun.csv')) ##0.28
pred3 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/2staged_cat_junhyuk_byme.csv'))
pred2 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/preprocessing_C_H_Catboost (1).csv'))
pred1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/12_10_준혁xgb_시드고정.csv'))

files = [pred1,pred2,pred3]
for i, f in enumerate(files):
  print(f"orignial score for file{i+1} : {nmae(answer.공급량.values, f.공급량.values)}")

orignial score for file1 : 0.11002126124008256
orignial score for file2 : 0.09679442197089647
orignial score for file3 : 0.10109239773303749


,일자|시간|구분,공급량
0,2019-01-01 01 A,2017.831563
1,2019-01-01 02 A,1750.736128
2,2019-01-01 03 A,1620.228004
3,2019-01-01 04 A,1673.513072
4,2019-01-01 05 A,1861.183495
...,...,...
15115,2019-03-31 20 H,428.106111
15116,2019-03-31 21 H,435.052081
15117,2019-03-31 22 H,419.090619
15118,2019-03-31 23 H,384.107653


In [66]:
## SIMPLE BLEND
all = []
pred = []
weight_list = [round(0.01*i, 3) for i in range(1,100)]
for w in weight_list:
     ensemble_pred = w * pred1.공급량.values + (1-w) * pred2.공급량.values
     ensemble_nmae = nmae( answer.공급량.values , ensemble_pred )
     all.append( ensemble_nmae )
     pred.append( ensemble_pred )
best_weight = weight_list[np.argmin( all )-1]
best_pred = pred[np.argmin( all )]
print(f"BEFORE1 : {nmae( answer.공급량.values , pred1.공급량.values )}")
print(f"BEFORE2 : {nmae( answer.공급량.values , pred2.공급량.values )}")
print(f"AFTER : {best_weight, all[np.argmin( all )]}")

BEFORE1 : 0.10109239773303749
BEFORE2 : 0.09676760309480435
AFTER : (0.37, 0.0920628806044091)


In [104]:
### CHEAT tHEN BLEND
pd.options.mode.chained_assignment = None 

print("CHEATING")
agg_weight1, agg_weight2 = [],[]
for i, f in enumerate(files):
  agg_pred = []
  agg_score = []
  agg_weight = []
  val_score = []
  weight_list = [round(0.0001*i, 8) for i in range(8000,12000)]
  for gubun in [0,1,2,3,4,5,6]:
    score = []
    pred = []
    for w in weight_list:
        ensemble_pred = w * f[f.구분==gubun].공급량.values
        ensemble_nmae = nmae( answer[answer.구분==gubun].공급량.values , ensemble_pred)
        score.append( ensemble_nmae )
        pred.append( ensemble_pred )
    best_weight = weight_list[np.argmin( score )-1]
    best_pred = pred[np.argmin( score )]
    agg_weight.append(best_weight)
    agg_pred.append(best_pred)
    agg_score.append(score[np.argmin( score )])
  print("=====================================================")
  print(f"FILE {i+1} BEFORE : {nmae( answer.공급량.values , f.공급량.values )}")
  print(f"BEST WEIGHT : {agg_weight}, SCORE : {np.array(agg_score).mean()} ")
  print("DOING VALIDATION")
  for gubun in [0,1,2,3,4,5,6]:
    ensemble_pred = agg_weight[gubun] * f[f.구분==gubun].공급량.values
    f.loc[f.구분==gubun,'공급량'] = ensemble_pred
    val_score.append(nmae( answer[answer.구분==gubun].공급량.values , ensemble_pred))
  print(f"RESULT : {np.array(val_score).mean()}")
  print("=====================================================")
  if i==0: agg_weight1=agg_weight
  if i==1: agg_weight2=agg_weight

print("BLENDING")
all = []
pred = []
weight_list = [round(0.001*i, 3) for i in range(1,1000)]
for w in weight_list:
     ensemble_pred = w * pred1.공급량.values + (1-w) * pred2.공급량.values
     ensemble_nmae = nmae( answer.공급량.values , ensemble_pred )
     all.append( ensemble_nmae )
     pred.append( ensemble_pred )
best_weight = weight_list[np.argmin( all )-1]
best_pred = pred[np.argmin( all )]
print(f"BEFORE1 : {nmae( answer.공급량.values , pred1.공급량.values )}")
print(f"BEFORE2 : {nmae( answer.공급량.values , pred2.공급량.values )}")
print(f"AFTER : {best_weight, all[np.argmin( all )]}")
print(f"Validation: {nmae(answer.공급량.values, best_pred)}")

CHEATING
FILE 1 BEFORE : 0.10997523352518355
BEST WEIGHT : [0.9072, 0.9344, 0.968, 0.9418, 0.9271, 0.9958, 0.9172], SCORE : 0.0940425042305448 
DOING VALIDATION
RESULT : 0.09404257947969195
FILE 2 BEFORE : 0.09676760309480435
BEST WEIGHT : [0.9544, 0.9641, 0.9855, 0.9718, 0.973, 1.0043, 0.9507], SCORE : 0.09265758762301306 
DOING VALIDATION
RESULT : 0.09265764222691462
BLENDING
BEFORE1 : 0.09404257947969197
BEFORE2 : 0.09265764222691461
AFTER : (0.492, 0.08048437693493583)
Validation: 0.08048437693493583


In [133]:
answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
pred3 = process_submission(pd.read_csv('/content/준우.csv'))
pred2 = process_submission(pd.read_csv(('/content/동주.csv')))
pred1 = process_submission(pd.read_csv('/content/준혁.csv'))
files = [pred1, pred2, pred3]

In [134]:
### CHEAT tHEN BLEND
pd.options.mode.chained_assignment = None 

print("CHEATING")
agg_weight1, agg_weight2, agg_weight3 = [],[],[]
for i, f in enumerate(files):
  agg_pred = []
  agg_score = []
  agg_weight = []
  val_score = []
  weight_list = [round(0.0001*i, 8) for i in range(8000,12000)]
  for gubun in [0,1,2,3,4,5,6]:
    score = []
    pred = []
    for w in weight_list:
        ensemble_pred = w * f[f.구분==gubun].공급량.values
        ensemble_nmae = nmae( answer[answer.구분==gubun].공급량.values , ensemble_pred)
        score.append( ensemble_nmae )
        pred.append( ensemble_pred )
    best_weight = weight_list[np.argmin( score )-1]
    best_pred = pred[np.argmin( score )]
    agg_weight.append(best_weight)
    agg_pred.append(best_pred)
    agg_score.append(score[np.argmin( score )])
  print("=====================================================")
  print(f"FILE {i+1} BEFORE : {nmae( answer.공급량.values , f.공급량.values )}")
  print(f"BEST WEIGHT : {agg_weight}, SCORE : {np.array(agg_score).mean()} ")
  print("DOING VALIDATION")
  for gubun in [0,1,2,3,4,5,6]:
    ensemble_pred = agg_weight[gubun] * f[f.구분==gubun].공급량.values
    f.loc[f.구분==gubun,'공급량'] = ensemble_pred
    val_score.append(nmae( answer[answer.구분==gubun].공급량.values , ensemble_pred))
  print(f"RESULT : {np.array(val_score).mean()}")
  print("=====================================================")
  if i==0: agg_weight1=agg_weight
  if i==1: agg_weight2=agg_weight
  if i==2: agg_weight3=agg_weight

# print("BLENDING")
# all = []
# pred = []
# weight_list = [round(0.001*i, 3) for i in range(1,1000)]
# for w in weight_list:
#      ensemble_pred = w * pred1.공급량.values + (1-w) * pred2.공급량.values
#      ensemble_nmae = nmae( answer.공급량.values , ensemble_pred )
#      all.append( ensemble_nmae )
#      pred.append( ensemble_pred )
# best_weight = weight_list[np.argmin( all )-1]
# best_pred = pred[np.argmin( all )]
# print(f"BEFORE1 : {nmae( answer.공급량.values , pred1.공급량.values )}")
# print(f"BEFORE2 : {nmae( answer.공급량.values , pred2.공급량.values )}")
# print(f"AFTER : {best_weight, all[np.argmin( all )]}")
# print(f"Validation: {nmae(answer.공급량.values, best_pred)}")
########################################################################################################
print("BLENDING")
all = []
pred = []
helper = 0
best = 10
weight_list = [round(0.01*i, 6) for i in range(1,1000)]
for w1 in weight_list:
  for w2 in weight_list:
     ensemble_pred = w1 * pred1.공급량.values + w2 * pred2.공급량.values + (1-w1-w2) * pred3.공급량.values
     ensemble_nmae = nmae( answer.공급량.values , ensemble_pred )
     if best > ensemble_nmae:
      best = ensemble_nmae
      helper = [w1,w2]
      pred = ensemble_pred

print(f"BEFORE1 : {nmae( answer.공급량.values , pred1.공급량.values )}")
print(f"BEFORE2 : {nmae( answer.공급량.values , pred2.공급량.values )}")
print(f"BEFORE3 : {nmae( answer.공급량.values , pred3.공급량.values )}")
print(f"AFTER : {[helper[0], helper[1],(1-np.array(helper).sum())], best}")
print(f"Validation: {nmae(answer.공급량.values, pred)}")

CHEATING
FILE 1 BEFORE : 0.09891493467577182
BEST WEIGHT : [1.1037, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], SCORE : 0.0942056827971598 
DOING VALIDATION
RESULT : 0.09420573024727916
FILE 2 BEFORE : 0.0970761813936346
BEST WEIGHT : [1.0531, 1.0286, 1.0232, 1.0307, 1.0455, 1.0106, 1.0429], SCORE : 0.09263713764290156 
DOING VALIDATION
RESULT : 0.09263719559960984
FILE 3 BEFORE : 0.09278685737422276
BEST WEIGHT : [1.0205, 1.0133, 1.011, 0.9984, 1.0415, 0.9945, 1.0318], SCORE : 0.09113498738240021 
DOING VALIDATION
RESULT : 0.09113505699116566
BLENDING
BEFORE1 : 0.09420573024727916
BEFORE2 : 0.09263719559960984
BEFORE3 : 0.09113505699116566
AFTER : ([0.41, 0.44, 0.15000000000000002], 0.08056186816781945)
Validation: 0.08056186816781945


In [129]:
answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
ppp1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/2staged_cat_junhyuk_byme.csv'))
ppp2 = process_submission(pd.read_csv(('/content/준우.csv'))) 
nmae(answer.공급량.values, ppp1.공급량.values), nmae(answer.공급량.values, ppp2.공급량.values), nmae(ppp1.공급량.values, ppp2.공급량.values)

(0.10109239773303749, 0.09278685737422276, 0.06022676171829446)

In [131]:
answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
ppp1 = process_submission(pd.read_csv('/content/preprocessing_C_H_Catboost (1).csv'))
ppp2 = process_submission(pd.read_csv('/content/동주.csv'))
nmae(answer.공급량.values, ppp1.공급량.values), nmae(answer.공급량.values, ppp2.공급량.values), nmae(ppp1.공급량.values, ppp2.공급량.values)

(0.09679442197089647, 0.0970761813936346, 0.05911239663417231)

In [132]:
answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
ppp1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/12_10_준혁xgb_시드고정.csv'))
ppp2 = process_submission(pd.read_csv('/content/준혁.csv'))
nmae(answer.공급량.values, ppp1.공급량.values), nmae(answer.공급량.values, ppp2.공급량.values), nmae(ppp1.공급량.values, ppp2.공급량.values)

(0.11002126124008256, 0.09891493467577182, 0.07128326100336234)

In [124]:
answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
pred3 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/2staged_cat_junhyuk_byme.csv'))
pred2 = process_submission(pd.read_csv(('/content/preprocessing_C_H_Catboost (1).csv')))
pred1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/12_10_준혁xgb_시드고정.csv'))

files = [pred1,pred2,pred3]
for i, f in enumerate(files):
  print(f"orignial score for file{i+1} : {nmae(answer.공급량.values, f.공급량.values)}")
pred_df_list = [pred1, pred2, pred3]
best_weight =[
 [0.9061, 0.9373, 0.9634, 0.9419, 0.9311, 0.9934, 0.9129], 
 [0.9542, 0.9638, 0.9862, 0.9711, 0.9734, 1.0037, 0.9515],
 [0.9236, 0.9476, 0.9733, 0.9368, 0.969, 0.986, 0.9415]
]
for i, f in enumerate(pred_df_list):
  for gubun in [0,1,2,3,4,5,6]:
    ensemble_pred = best_weight[i][gubun] * f[f.구분==gubun].공급량.values
    f.loc[f.구분==gubun,'공급량'] = ensemble_pred
print(nmae(answer.공급량.values,pred1.공급량.values))
print(nmae(answer.공급량.values,pred2.공급량.values))
print(nmae(answer.공급량.values,pred3.공급량.values))
ensemble_pred = (0.41) * pred1.공급량.values + (0.44) * pred2.공급량.values + (1-0.41-0.44) * pred3.공급량.values
print(nmae(answer.공급량.values,ensemble_pred))

orignial score for file1 : 0.11002126124008256
orignial score for file2 : 0.09679442197089647
orignial score for file3 : 0.10109239773303749
0.09420575239867135
0.09265224137723971
0.09080348813189255
0.08053127313416897


In [120]:
best_weight[0]

[0.9061, 0.9373, 0.9634, 0.9419, 0.9311, 0.9934, 0.9129]

In [126]:
answer = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv'))
pred3 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/2staged_cat_junhyuk_byme.csv'))
pred2 = process_submission(pd.read_csv(('/content/preprocessing_C_H_Catboost (1).csv')))
pred1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/12_10_준혁xgb_시드고정.csv'))

files = [pred1,pred2,pred3]
for i, f in enumerate(files):
  print(f"orignial score for file{i+1} : {nmae(answer.공급량.values, f.공급량.values)}")


orignial score for file1 : 0.11002126124008256
orignial score for file2 : 0.09679442197089647
orignial score for file3 : 0.10109239773303749


In [114]:
nmae(answer.공급량.values, pred)

0.08053127313416897

In [115]:
PATH = "/content/drive/MyDrive/dacon/gas/data"
submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
submission['공급량'] = pred
submission.to_csv(os.path.join("/content/drive/MyDrive/dacon/gas/submission", "final_ensemble_best.csv"), index=False)

In [95]:
### 일단 위에 CHEATING 부분에서 값들을 이미 바꿔놨고, BLENDING에서 파일을 섞는 것이기 때문에
### 아래 코드는 SIMPLE BLENDING 대신 월별 BLENDING 적용한 코드이다
agg_pred = []
agg_score = []
agg_weight = []
val_score = []
weight_list = [round(0.001*i, 8) for i in range(1,10000)]
for m in [1,2,3]:
  score = []
  pred = []
  for w in weight_list:
      ensemble_pred = w * pred1[(pred1.연월일.dt.month==m)].공급량.values + (1-w) * pred2[(pred2.연월일.dt.month==m)].공급량.values
      ensemble_nmae = nmae( answer[(answer.연월일.dt.month==m)].공급량.values , ensemble_pred)
      score.append( ensemble_nmae )
      pred.append( ensemble_pred )
  best_weight = weight_list[np.argmin( score )-1]
  best_pred = pred[np.argmin( score )]
  agg_weight.append(best_weight)
  agg_pred.append(best_pred)
  agg_score.append(score[np.argmin( score )])
print(f"BEFORE1 : {nmae( answer.공급량.values , pred1.공급량.values )}")
print(f"BEFORE2 : {nmae( answer.공급량.values , pred2.공급량.values )}")
print("=====================================================")
print(f"BEST WEIGHT : {agg_weight} SCORE : {np.array(agg_score).mean()} ")
print("=====================================================")
print("DOING VALIDATION")
i=0
for m in [1,2,3]:
  ensemble_pred = agg_weight[i] * pred1[(pred1.연월일.dt.month==m)].공급량.values + (1-agg_weight[i]) * pred2[(pred2.연월일.dt.month==m)].공급량.values
  val_score.append(nmae( answer[(answer.연월일.dt.month==m)].공급량.values , ensemble_pred))
  i+=1
print(f"RESULT : {np.array(val_score).mean()}")

BEFORE1 : 0.09080348813189255
BEFORE2 : 0.09265764222691461
BEST WEIGHT : [0.43, 1.652, 0.287] SCORE : 0.07646971903970272 
[1.1999, 1.1999, 1.1999] 0.09547891461461167
DOING VALIDATION
RESULT : 0.07646976525090085


In [99]:
i=0
to_submit=[]
for m in [1,2,3]:
  ensemble_pred = agg_weight[i] * pred1[(pred1.연월일.dt.month==m)].공급량.values + (1-agg_weight[i]) * pred2[(pred2.연월일.dt.month==m)].공급량.values
  val_score.append(nmae( answer[(answer.연월일.dt.month==m)].공급량.values , ensemble_pred))
  to_submit.append(ensemble_pred)
  i+=1
print(f"RESULT : {np.array(val_score).mean()}")
PATH = "/content/drive/MyDrive/dacon/gas/data"
submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
submission['공급량'] = to_submit
submission.to_csv(os.path.join("/content/drive/MyDrive/dacon/gas/submission", "2file_ensemble_m.csv"), index=False)

RESULT : 0.07646976525090084


ValueError: ignored

In [98]:
len(ensemble_pred)

5208

In [93]:
### 일단 위에 CHEATING 부분에서 값들을 이미 바꿔놨고, BLENDING에서 파일을 섞는 것이기 때문에
### 아래 코드는 SIMPLE BLENDING 대신 월별 BLENDING 적용한 코드이다
agg_pred = []
agg_score = []
agg_weight = []
val_score = []
weight_list = [round(0.001*i, 8) for i in range(0,1000)]
for month in [1,2,3]:
  score = []
  pred = []
  for w in weight_list:
      ensemble_pred = w * pred1[(pred1.연월일.dt.month==m)].공급량.values + (1-w) * pred2[(pred2.연월일.dt.month==m)].공급량.values
      ensemble_nmae = nmae( answer[answer.연월일.dt.month==m].공급량.values , ensemble_pred)
      score.append( ensemble_nmae )
      pred.append( ensemble_pred )
  best_weight = weight_list[np.argmin( score )-1]
  best_pred = pred[np.argmin( score )]
  print(f"FOR MONTH {month}")
  print(f"BEFORE : {nmae( answer[answer.연월일.dt.month==m].공급량.values , pred1[(pred1.연월일.dt.month==m)].공급량.values )}")
  print(f"BEFORE : {nmae( answer[answer.연월일.dt.month==m].공급량.values , pred2[(pred2.연월일.dt.month==m)].공급량.values )}")
  print(f"AFTER : {best_weight, score[np.argmin( score )]}")
  print(f"VAL   : {nmae(answer[answer.연월일.dt.month==m].공급량.values, best_pred)}")
  agg_weight.append(best_weight)
  agg_pred.append(best_pred)
  agg_score.append(score[np.argmin( score )])
print("=====================================================")
print(f"BEST WEIGHT : {agg_weight}, SCORE : {np.array(agg_score).mean()} ")
print("=====================================================")
print("DOING VALIDATION")
for m in [1,2,3]:
  ensemble_pred = agg_weight[m-1] * pred1[(pred1.연월일.dt.month==m)].공급량.values + (1-(agg_weight[m-1])) * pred2[(pred2.연월일.dt.month==m)].공급량.values
  val_score.append(nmae( answer[answer.연월일.dt.month==m].공급량.values , ensemble_pred))
print(f"RESULT : {np.array(val_score).mean()}")

FOR MONTH 1
BEFORE : 0.12393004371966593
BEFORE : 0.09914915723344628
AFTER : (0.287, 0.0940183375402021)
VAL   : 0.0940183375402021
FOR MONTH 2
BEFORE : 0.12393004371966593
BEFORE : 0.09914915723344628
AFTER : (0.287, 0.0940183375402021)
VAL   : 0.0940183375402021
FOR MONTH 3
BEFORE : 0.12393004371966593
BEFORE : 0.09914915723344628
AFTER : (0.287, 0.0940183375402021)
VAL   : 0.0940183375402021
BEST WEIGHT : [0.287, 0.287, 0.287], SCORE : 0.0940183375402021 
DOING VALIDATION
RESULT : 0.08729100699562076


In [75]:
val_score=[]
i=0
for m in [1,2,3]:
  ensemble_pred = best_w[i] * pred1[(pred1.연월일.dt.month==m)].공급량.values + (1-best_w[i]) * pred2[(pred2.연월일.dt.month==m)].공급량.values
  val_score.append(nmae( answer[(answer.연월일.dt.month==m)].공급량.values , ensemble_pred))
  i+=1
print(f"RESULT : {np.array(val_score).mean()}")

RESULT : 0.09547891461461167


In [117]:
p1 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/preprocessing_C_H_Catboost.csv'))
p2 = process_submission(pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/preprocessing_C_H_Catboost (1).csv'))

nmae(p1.공급량.values,p2.공급량.values)

0.002455361248389918

# 밑으로는 안해봄 . 0.05파일이 있는이상 무쓸모

In [ ]:
###방법 2 - euclidean distance와 각각 element 차이를 활용
## diff_list는 ans element와 각 pred element차이
## dist는 euclidean distnace between list pred and list ans
def weight_cal(ans,pred):
  diff_list = [ans[i]-pred[i] for i in range(0, len(ans))]
  dist = np.linalg.norm(np.array(ans)-np.array(pred))
  return diff_list, dist

diff_list1, dist1 = weight_cal(ans, pred1)
diff_list2, dist2 = weight_cal(ans, pred2)

#전체 리스트의 eucledian distance를 weight으로 쓴다
submit = dist1/(dist1+dist2+dist3)*pred1 + dist2/(dist1+dist2+dist3)*pred2 + dist3/(dist1+dist2+dist3)*pred3
#아니면 각 element의 차이를 weight으로 쓴다
submit = diff_list1/(dist1+dist2+dist3)*pred1 ....
